<left>FINM 32000 - Numerical Methods</left>
<left>Spring 2023</left>
<br>
<h1><center> Final Exam </center></h1>
<center>18:00 - 21:00 [CST] May 24th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

### Imports

In [1]:
import numpy as np
from scipy.stats import norm

### Helper-Functions

In [2]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma


In [3]:
class Call:

    def __init__(self,K,T):
        self.K=K
        self.T=T

    #You will probably want to insert another function here, to support the control variate calculation in (b)
    def BSPrice(self, dynamics):
        # Black-Scholes Method:
        d_1 = (np.log(dynamics.S0/self.K) +
               ((dynamics.r) + dynamics.sigma ** 2 / 2) * self.T) / (dynamics.sigma * np.sqrt(self.T))
        d_2 = d_1 - (dynamics.sigma * np.sqrt(self.T))
        return norm.cdf(d_1) * dynamics.S0 - norm.cdf(d_2) * self.K * np.exp(-dynamics.r * self.T)

In [4]:
class CallOnAverage:

    def __init__(self,K,T0,T1):
        self.K = K
        self.T0 = T0
        self.T1 = T1

In [15]:
class MC:

    def __init__(self,M,seed,useControl):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible
        self.useControl = useControl

    def randomLogreturn(self,dynamics,deltat):
        return (dynamics.r-dynamics.sigma**2/2)*deltat + dynamics.sigma*np.sqrt(deltat)*self.rng.normal(size=self.M)

    def price_CallOnAverage_GBM(self,contract,dynamics):
    
        #You fill this in.  If self.useControl is False, then this should do part (a), otherwise it should do part (b)
        X_late = self.randomLogreturn(dynamics, contract.T1)
        X_middle = self.randomLogreturn(dynamics, contract.T0)
        payoff = dynamics.S0 * (np.exp(X_late) + np.exp(X_middle))/2 - contract.K
        discount = np.exp(-dynamics.r * contract.T1)
        price = discount * np.maximum(payoff, 0.0)

        if self.useControl:
            while True:
                T = 1
                K = 10
                call_payoff_star = dynamics.S0 * np.exp(self.randomLogreturn(dynamics, T)) - K
                if np.cov(call_payoff_star, payoff)[0, 1] > 0.9:
                    break
                T += 0.001
                K += 0.001
            call_price_star = discount * np.maximum(call_payoff_star, 0.0)
            control_call = Call(K, T)
            C_star = control_call.BSPrice(dynamics)
            # MLE beta
            beta_star = np.cov(price, call_price_star)[0, 1]/np.var(call_price_star)
            price = price + beta_star * (C_star - call_price_star)

        standard_error = np.std(price)
        price = np.mean(price)

        return(price, standard_error)


# Problem 1.

The payoff of the European $K$-strike vanilla call is:

$$
(S_T - K)^+
$$

It was given that $S_T$ follows a distribution with density function $p$
and cumulative distribution function $F$.

For the time $0$ price, the payoff at time $T$ has to be discounted by
$e^{-rT}$ and this value was given in the question as $0.8$.

Therefore, the value of the European $K$-strike vanilla call is:

$$
\begin{aligned}
C(K) &= e^{-rT} \left(
\int_K^{\infty} (S_T - K) p(S_T) dS_T
\right) \\
&= e^{-rT} \int_K^{\infty} S_T p(S_T) dS_T -
e^{-rT} K \int_K^{\infty} p(S_T)dS_T \\
&= e^{-rT} \int_K^{\infty} S_T p(S_T) dS_T -
e^{-rT} K (1 - F(K))
\end{aligned}
$$

It was also given that $C(100) = 8$, $p(100) = 0.15$, and $F(100) = 0.5$

## (a)

Using a second-order Taylor expansion, $C(100 + h)$ can be approximated as:

$$
C(100 + h) \approx C(100) + h C'(100) + \frac{1}{2} h^2 C''(100)
$$

Therefore, we first need to attain $C'(K)$ and $C''(K)$ from the above equation.

From the set-up above,

$$
\begin{aligned}
C'(K) =
\frac{\delta}{\delta K} C(K) &= -e^{-rT} K p(K) -
e^{-rT}((1 - F(K)) - KF'(K)) \\
&= -e^{-rT} K p(K) - e^{-rT} (1 - F(K) - K p(K)) \\
&(\because F'(x) = p(x)) \\
&= e^{-rT} \left(
-K p(K) - 1 + F(K) + K p(K)
\right) \\
&= e^{-rT} (F(K) - 1)
\end{aligned}
$$

Moreover,

$$
\begin{aligned}
C''(K) =
\frac{\delta^2}{\delta K^2} C(K) &= \frac{\delta}{\delta K} C'(K) \\
&= e^{-rT} F'(K) \\
&= e^{-rT} p(K) \\
&(\because F'(x) = p(x))
\end{aligned}
$$

Using the above, the second-order Taylor expansion becomes:

$$
C(100 + h) \approx C(100) + h e^{-rT} (F(100) - 1) + \frac{1}{2} h^2 e^{-rT} p(K)
$$

Substituting the numerical values gives:

$$
C(99.5) \approx 8 + (-0.5) \times 0.8 \times (0.5 - 1) +
\frac{1}{2} (-0.5)^2 \times 0.8 \times 0.15 = 8.215
$$

## (b)

For a binary put with strike $K$ and expiry $T$, the payoff is $1$ when $S_T < K$.

With our probability density function, the price of the binary put can be formulated as:

$$
Binary Put(K) = e^{-rT} \int_0^K 1 \cdot p(S_T) dS_T
= e^{-rT} F(K)
$$

If we substitute the numerical values, the question is asking for $0.8 \cdot F(101)$.
We would need to approximate $F(101)$ since only $F(100)$ is known.

Using 1st degree Taylor approximation:

$$
F(101) = F(100) + 1 \cdot F'(100) + O(K)^2
$$

We know that $F'(100) = p(100) = 0.15$.

Therefore, the approximation yields:

$$
Binary Put(101) = 0.8 \cdot F(101)
\approx 0.8 \cdot (0.5 + 0.15)
= 0.52
$$

## (c)

With the given properties,

$$
\begin{aligned}
\mathbf{P}(99 < S_T < 101) &=
\int_{99}^{101} p(S_T) dS_t \\
&= \int_{0}^{101} p(S_T) dS_t -
\int_{0}^{99} p(S_T) dS_t \\
&= F(101) - F(99)
\end{aligned}
$$

Similar to the FD scheme, we can make the rough estimation that:

$$
F'(100) \approx \frac{F(101) - F(99)}{2}
$$

Here we know that $F'(100) = p(100) = 0.15$.

Therefore,

$$
\mathbf{P}(99 < S_T < 101) = F(101) - F(99) \approx 0.15 \times 2 = 0.3
$$

# Problem 2.



## (a)

Using the standard explicit FD scheme:

$$
\frac{\delta}{\delta t} C(15.0, 0.1) \approx
\frac{\mathbf{C}(15.0, 0.1) - \mathbf{C}(15.0, 0)}{0.1}
$$

$$
\frac{\delta^2}{\delta x^2} C(15.0, 0.1) \approx
\frac{\mathbf{C}(15.5, 0.1) - 2\mathbf{C}(15.0, 0.1) + \mathbf{C}(14.5, 0.1)}{0.5^2}
$$

Therefore, the PDE becomes:

$$
\frac{\mathbf{C}(15.0, 0.1) - \mathbf{C}(15.0, 0)}{0.1} +
0.75 \frac{\mathbf{C}(15.5, 0.1) - 2\mathbf{C}(15.0, 0.1) + \mathbf{C}(14.5, 0.1)}{0.5^2}
= 0
$$

If we substitute the numerical values:

$$
\begin{aligned}
& \frac{1.05 - \mathbf{C}(15.0, 0)}{0.1} +
0.75 \frac{1.16 - 2 \cdot 1.05 + 1.04}{0.5^2}
= 0 \\
& \mathbf{C}(15.0, 0) = 0.1(
10.5 + 3 (1.16 - 2.10 + 1.04)) \\
&= 0.1(10.5 + 3 \cdot 0.1) \\
&= 0.1 \cdot 10.8 \\
& \therefore
\mathbf{C}(15.0, 0) = 1.08
\end{aligned}
$$

## (b)

Keeping the $\frac{\delta^2}{\delta x^2} C(15.0, 0.1)$ term same, but now with a different
$\frac{\delta}{\delta t} C(15.0, 0.1)$ term.

The new $\frac{\delta}{\delta t} C(15.0, 0.1)$ term can be estimated as:

$$
\frac{\delta}{\delta t} C(15.0, 0.1) \approx
\frac{\mathbf{C}(15.0, 0.2) - \mathbf{C}(15.0, 0)}{0.2}
$$

Therefore, the PDE becomes:

$$
\frac{\mathbf{C}(15.0, 0.2) - \mathbf{C}(15.0, 0)}{0.2} +
0.75 \frac{\mathbf{C}(15.5, 0.1) - 2\mathbf{C}(15.0, 0.1) + \mathbf{C}(14.5, 0.1)}{0.5^2}
= 0
$$

If we substitute the numerical values:

$$
\begin{aligned}
& \frac{1.01 - \mathbf{C}(15.0, 0)}{0.2} +
0.75 \frac{1.16 - 2 \cdot 1.05 + 1.04}{0.5^2}
= 0 \\
& \mathbf{C}(15.0, 0) = 0.2(
5.05 + 3 (1.16 - 2.10 + 1.04)) \\
&= 0.2(5.05 + 3 \cdot 0.1) \\
&= 0.2 \cdot 5.35 \\
& \therefore
\mathbf{C}(15.0, 0) = 1.07
\end{aligned}
$$

## (c)

Scheme (a) does guarantee the stability property

Scheme (b) does not guarantee the stability property.

# Problem 3.

In [6]:
p3dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [7]:
p3contract = CallOnAverage(K=12,T0=0.5,T1=1.0)

## (a)

In [13]:
p3aMC = MC(M=100000,seed=0,useControl=False) 
(price3a, standard_error3a) = p3aMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(price3a, standard_error3a)

1.1389444203880121 3.079989098787988


## (b)

In [ ]:
p3bMC = MC(M=100000,seed=0,useControl=True)
(price3b, standard_error3b) = p3bMC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(price3b, standard_error3b)

# Problem 4.

## (a)

Supposing that the game is played, let's calculate the optimized profit.

### Case 1: First card Red

You will have a net profit of $-1$ and will be forced to stop the game.

The probability of this happening is $\frac{1}{2}$.

### Case 2: First card Black

You will have a net profit of $1$

The probability of drawing a black card is $\frac{1}{2}$.

We will need to investigate further what the optimal choice will be beyond this

#### Case 2-1: Black-Black

You will have a net profit of $2$ and the optimal choice would be to
quit since there are only Red cards left.

The probability of this happening after drawing the black card is $\frac{1}{3}$.

#### Case 2-2: Black-Red

At this point your net profit is $0$.

The probability of this happening after drawing the black card is $\frac{2}{3}$.

If you decide to continue, there is a $\frac{1}{2}$ chance that you will gain 1 dollar
and a $\frac{1}{2}$ chance that you will lose 1 dollar and will be forced to quit.
This would mean that the optimal choice is either continue or quit.

The expected return from continuing the game is $0$

Together, the expected profit of playing more cards after drawing the first black card is
$2 \times \frac{1}{3} + 0 \times \frac{2}{3} = \frac{2}{3}$.

Since the expected profit from continuing to play the game is positive,
the optimal choice would be to continue drawing after the first black card.

This would conclude the overall expected profit from playing the game to be:

$$
-1 \times \frac{1}{2} +
2 \times \frac{1}{2} \times \frac{1}{3} +
0 \times \frac{1}{2} \times \frac{2}{3} =
-\frac{1}{6}
$$

Since the expected profit from optimally playing this game is negative,
**you should not play this game**

## (b)

Let's calculate again what the expected profit would be if the
game is played optimally

### Case 1: First card Red

Your net profit at this point will be $-1$.

The probability of this occurring is $\frac{1}{2}$

This time you would not be forced to quit, but you would need to
calculate what the optimal choice is at this point

#### Case 1-1: Red - Red

The probability of drawing the second red card is $\frac{1}{3}$.

You will be forced to stop the game and the net profit will be $-2$.

#### Case 1-2: Red - Black

The probability of drawing the black card in the second is $\frac{2}{3}$.

Your net profit at this point is $0$.

It is optimal to continue playing since you won't be forced to quit
even if you draw all $4$ cards (with net profit $0$) and there is a
probability of having a positive net profit if your third card is also
black (at which point it would be optimal to quit the game).

The expected net profit from the optimal choice after drawing Red-Black is:

$$
1 \times \frac{1}{2} +
0 \times \frac{1}{2} = \frac{1}{2}
$$

Together, the net profit from continue playing after drawing the first Red is:

$$
-1 \times \frac{1}{3} +
\frac{1}{2} \times \frac{2}{3} =
0
$$

The optimal strategy is indifferent to continue playing or quiting after the first Red

### Case 2: First card Black

Your net profit at this point will be $1$.

The probability of this occurring is $\frac{1}{2}$

It will be optimal to continue playing since there is probability of having positive
profit of doing so and you will never be forced to quit.

If the second card is also black you quit the game with net profit $2$.

If the second card is red, you continue playing.
This is because if the third card is black you quit the game with net profit $1$.
Nevertheless, if the third card is red you can play the fourth card and end the
game with net profit $0$.

From Case 1 and Case 2, the expected net profit is:

$$
-1 \times \frac{1}{2} +
2 \times \frac{1}{2} \times \frac{1}{3} +
1 \times \frac{1}{2} \times \frac{2}{3} \times \frac{1}{2} +
0 \times \frac{1}{2} \times \frac{2}{3} \times \frac{1}{2} =
0
$$

Therefore, if I play, the expected optimized profit is $0$.
There would be no difference in playing or not playing based on the expected optimized profit.
The actual decision would depend on your preference (i.e., statistically there is no
dominating strategy or difference).

# Problem 5.

## (a)

By definition:

$$
\psi_{a, b}(w) := \mathbf{E}[e^{iw(aR_1 + bR_2)}]
$$

Furthermore,

$$
\begin{aligned}
\mathbf{E}[e^{iw(aR_1 + bR_2)}] &=
\mathbf{E}[e^{iawR_1 + ibwR_2}] \\
&= F(aw, bw)
\end{aligned}
$$

Therefore, $\psi_{a, b}(w)$ can be expressed as $F(aw, bw)$.

## (b)

We know that $\log(X/Y) = \log(X) - \log(Y)$.

Therefore, the characteristic function of $\log(X/Y)$ would be
equivalent to $F(1, -1)$

Using this we may conclude that:

$$
\begin{aligned}
\mathbf{E} 1_{X/Y > K} &=
\mathbf{P} (X/Y > K) \\
&= \mathbf{P} (\log(X) - \log(Y) > \log(K)) \\
&= \frac{1}{2} - \frac{1}{\pi}
\int_0^{\infty} Re \left[\frac{F_{1, 1}(z)}{iz} e^{-iz \log(K)} \right] dz
\end{aligned}
$$